In [2]:
from selenium.webdriver.common.by import By
from __future__ import unicode_literals
from fake_useragent import UserAgent
from selenium import webdriver
from bs4 import BeautifulSoup
from yt_dlp import YoutubeDL

import urllib.request
import time
import os

# Step 1: Get Songs

In [ ]:
url = "https://www.billboard.com/charts/greatest-hot-100-singles/"

# writes raw HTML to file
with open("billboard.txt", "w") as file:
    file.write(urllib.request.urlopen(url).read().decode("utf-8"))        

# parses raw HTML
with open("billboard.txt", "r") as file:
    html = file.read()
soup = BeautifulSoup(html, "html.parser")

sp = soup.get_text().replace("\n", "")
print(sp)

In [ ]:
# [rank-1] [song title] [artist name]
rankings = []

i = 1
while len(rankings) < 100:
    ind = sp.find(f"\t{i}\t")
    rest = sp[ind:].lstrip("\t")
    line = rest[:rest.find(f"{i+1}")]
    
    num = line[:len(str(i))]
    
    # line now starts at song title
    line = line[len(str(i)):].lstrip("\t")
    songTitle = line[:line.find("\t")]
    
    # line now starts at artist name
    line = line[len(songTitle):].lstrip("\t")
    artistName = line[:line.find("\t")]

    rankings.append([songTitle, artistName])

    i += 1

# for i, entry in enumerate(rankings):
#     song, artist = entry
#     print(f"{i}: {song}\t{artist}")
rankings[-1][1] = "Carly Simon" # fixing an error
print(rankings[-1])

## Web Scraping

In [ ]:
# start driver
options = webdriver.SafariOptions()

ua = UserAgent(browsers=["safari"])
options.add_argument(f"user-agent={ua.random}")
options.add_argument("--incognito")

driver = webdriver.Safari(options=options)
driver.maximize_window()

### Get link to YouTube video

In [39]:
# test: get song from top of billboard top 100
song, artist = rankings[78]
driver.get(f"https://www.youtube.com/results?search_query={song}+{artist}")

In [ ]:
# get link to video
time.sleep(20)
href = None
videos = driver.find_elements(
            By.TAG_NAME, 
            "ytd-thumbnail"
        )
while len(videos) > 0 and not href:
    href = videos[0].get_attribute("href")
    videos = videos[1:]
    
if not href:
    videos = driver.find_elements(
        By.ID, 
        "video-title"
    )
    while len(videos) > 0 and not href:
        href = videos[0].get_attribute("href")
        videos = videos[1:]
        
if not href:
    videos = driver.find_elements(
        By.ID, 
        "movie_player"
    )
    while len(videos) > 0 and not href:
        href = videos[0].get_attribute("href")
        videos = videos[1:]

print(href)

### Get key of song

In [ ]:
driver.get("https://tunebat.com")
time.sleep(10)

# type song and artist into search bar
search = driver.find_element(
    By.XPATH,
    "//input[@aria-label='Song search field']"
)
search.send_keys(f"{song} {artist}")

time.sleep(1)

# click search button
button = driver.find_element(
    By.XPATH,
    "//i[@aria-label='Search']"
)
button.click()

time.sleep(5)

# get info on song
song_info = driver.find_elements(
    By.CLASS_NAME,
    "lAjUd"
)
key, bpm = [item.text for item in song_info[:2]]
print(key, bpm)

### Download YouTube video using youtube_dl

In [ ]:
kebab_artist = '-'.join(artist.split())
kebab_song = '-'.join(song.split())
dl_path = os.join(os.getcwd(),
                  "songs",
                  str(kebab_artist),
                  str(kebab_song),
                  str(key),
                  str(bpm),
                  ".mp3"
                 )

with YoutubeDL({
     'extract_audio': True,
     'format': 'bestaudio',
     'outtmpl': dl_path
    }) as audio:
    
    audio.download(href)

# Step 1.1: Convert to .m4a (.mp3 is incompatible with librosa)

In [ ]:
# find which files did not convert to .m4a
converted = {}
need_to_convert = []

for file in os.listdir(os.path.join("instrumentals")):
    if file[0] != ".":
        filename = file[:-4]
        extension = file[-4:]
        if filename not in converted:
            converted.update({filename : [extension]})
        else:
            converted[filename].append(extension)
            
for key, value in converted.items():
    if len(value) < 2:
        print(f"{key} did not convert")
        need_to_convert.append(key+value[0])

In [ ]:
# convert remaining files in "/instrumentals" to .m4a
for file in need_to_convert:
    filepath = os.path.join(os.getcwd(), "instrumentals", file)
    os.system(f"ffmpeg -i {filepath} -c:a aac -b:a 192k {filepath[:-4]}.m4a")

In [ ]:
# convert all audio files in "/songs" from .mp3 to .m4a
for file in os.listdir(os.path.join("songs")):
    if file[0] != ".":
        filepath = os.path.join(os.getcwd(), "songs", file)
        os.system(f"ffmpeg -nostats -loglevel panic -i {filepath} -c:a aac -b:a 192k {filepath[:-4]}.m4a")

In [ ]:
# convert all audio files in "/instrumentals" from .mp3 to .m4a
for file in os.listdir(os.path.join("instrumentals")):
    if file[0] != ".":
        filepath = os.path.join(os.getcwd(), "instrumentals", file)
        os.system(f"ffmpeg -nostats -loglevel panic -i {filepath} -c:a aac -b:a 192k {filepath[:-4]}.m4a")